In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from preprocess import * 
from predict import *
from utils import *
from torchmetrics import WordErrorRate, CharErrorRate, MatchErrorRate

/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/said/anaconda3/envs/openmmlab/lib/python3.8/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [3]:
data_path = "/home/said/courses/Projetinfo/sports_scr/data/champions/natation.txt"
dic_names = get_champions_names(data_path)
scoreboard_path = "../data/natation/ROI/CLIP_visual_prompt/scoreboard.png"
default_visual_prompt = Image.open(scoreboard_path)
default_visual_prompt = np.array(default_visual_prompt)[:,:, 0:3]

In [4]:
def get_ground_truth(path):
    """Returns ground truth data

    Args:
        path (str): path to the ground truth txt file

    Returns:
        list: list of the ground truth data
    """

    with open(path, "r") as f:
        return [line.strip() for line in f.readlines()]
    
def compute_metrics(result, groundtruth):
    wer = WordErrorRate()
    cer = CharErrorRate()

    word_error_rate = 0
    character_error_rate = 0

    for yhat, y in zip(result, groundtruth):
        word_error_rate += wer(yhat, y)
        character_error_rate += cer(yhat, y)

    return word_error_rate/min(len(result), len(groundtruth)), character_error_rate/min(len(result), len(groundtruth))

In [5]:
data_path ="../data/champions/natation.txt"
dic_names = get_champions_names(data_path)

In [6]:
def process_img(img, orientation, clip):
    if clip:
        modified_img = extract_roi_clipseg_visual(img, prompt=default_visual_prompt, thresh=0.5)
        if orientation:
            modified_img = correct_orientation(modified_img)
        return modified_img
    else:
        if orientation:
            modified_img = correct_orientation(np.array(Image.open(img)))
            return modified_img
        else:
            return np.array(Image.open(img))

def evaluate(ocr_engine, orientation, clip, correct_names=False):
    total_wer, total_cer = 0, 0
    img_paths = create_images_path_list("../data/natation")
    for img in img_paths:
        processed_img = process_img(img, orientation, clip)

        predictions = infer(ocr_engine, processed_img)
        input_string = ' '.join(predictions)
        result = extract_names_scores(input_string, dic_names, correct_names=correct_names, min_edit_distance=3)

        groundtruth_path = img.split('.')
        groundtruth_path[-1] = ".txt"
        groundtruth_path = '..' + ''.join(groundtruth_path[2:])
        groundtruth = get_ground_truth(groundtruth_path)

        try:
            wer, cer = compute_metrics(result, groundtruth)
        except:
            wer, cer = 1, 1
        total_wer += wer
        total_cer += cer
    return total_wer/len(img_paths), total_cer/len(img_paths)

# docTR

## Base images

In [7]:
doctr_base_total_wer, doctr_base_total_cer = evaluate(ocr_engine='doctr', orientation=False, clip=False, correct_names=False)

In [8]:
doctr_base_total_wer, doctr_base_total_cer

(tensor(0.9926), tensor(0.8739))

## Base + correction d'orientation

In [37]:
doctr_orient_total_wer, doctr_orient_total_cer = evaluate(ocr_engine='doctr', orientation=True, clip=False, correct_names=False)

In [38]:
doctr_orient_total_wer, doctr_orient_total_cer

(tensor(0.7203), tensor(0.4842))

## CLIP images

In [39]:
doctr_clip_total_wer, doctr_clip_total_cer = evaluate(ocr_engine='doctr', orientation=False, clip=True, correct_names=False)

In [40]:
doctr_clip_total_wer, doctr_clip_total_cer

(tensor(0.7591), tensor(0.5711))

## CLIP + correction d'orientation

In [8]:
doctr_clip_orient_total_wer, doctr_clip_orient_total_cer = evaluate(ocr_engine='doctr', orientation=True, clip=True, correct_names=False)

In [9]:
doctr_clip_orient_total_wer, doctr_clip_orient_total_cer

(tensor(0.8448), tensor(0.6299))

# EasyOCR

## Base images

In [10]:
doctr_base_total_wer, doctr_base_total_cer = evaluate(ocr_engine='easyocr', orientation=False, clip=False, correct_names=False)

In [11]:
doctr_base_total_wer, doctr_base_total_cer

(tensor(0.7428), tensor(0.5751))

## Base + correction d'orientation

In [12]:
doctr_orient_total_wer, doctr_orient_total_cer = evaluate(ocr_engine='easyocr', orientation=True, clip=False, correct_names=False)

In [13]:
doctr_orient_total_wer, doctr_orient_total_cer

(tensor(0.6622), tensor(0.4801))

## CLIP images

In [14]:
doctr_clip_total_wer, doctr_clip_total_cer = evaluate(ocr_engine='easyocr', orientation=False, clip=True, correct_names=False)

In [15]:
doctr_clip_total_wer, doctr_clip_total_cer

(tensor(0.7407), tensor(0.5696))

## CLIP + correction d'orientation

In [16]:
doctr_clip_orient_total_wer, doctr_clip_orient_total_cer = evaluate(ocr_engine='easyocr', orientation=True, clip=True, correct_names=False)

In [17]:
doctr_clip_orient_total_wer, doctr_clip_orient_total_cer

(tensor(0.7957), tensor(0.6285))

# Pytesseract

## Base images

In [ ]:
doctr_base_total_wer, doctr_base_total_cer = evaluate(ocr_engine='pytesseract', orientation=False, clip=False)

In [ ]:
doctr_base_total_wer, doctr_base_total_cer

(tensor(0.7518), tensor(0.5522))

## Base + correction d'orientation

In [ ]:
doctr_orient_total_wer, doctr_orient_total_cer = evaluate(ocr_engine='pytesseract', orientation=True, clip=False)

In [ ]:
doctr_orient_total_wer, doctr_orient_total_cer

(tensor(0.7203), tensor(0.4842))

## CLIP images

In [ ]:
doctr_clip_total_wer, doctr_clip_total_cer = evaluate(ocr_engine='pytesseract', orientation=False, clip=True)

In [ ]:
doctr_clip_total_wer, doctr_clip_total_cer

(tensor(0.7591), tensor(0.5711))

## CLIP + correction d'orientation

In [ ]:
doctr_clip_orient_total_wer, doctr_clip_orient_total_cer = evaluate(ocr_engine='pytesseract', orientation=True, clip=True)

In [ ]:
doctr_clip_orient_total_wer, doctr_clip_orient_total_cer

(tensor(0.8448), tensor(0.6299))

# Experiments

In [11]:
import matplotlib.pyplot as plt
from typing import List, Tuple
import matplotlib.colors as mcolors



def evaluate_all_combinations() -> Tuple[List[float], List[float]]:
    wer_list = []
    cer_list = []
    ocr_engines = ["doctr", "easyocr", "pytesseract"]
    orientations = [True, False]
    clips = [True, False]
    correct_names_options = [True, False]

    # Evaluate all combinations of the parameters
    for engine in ocr_engines:
        for orientation in orientations:
            for clip in clips:
                for correct_names in correct_names_options:
                    print(f"------ engine = {engine}, orientation={orientation}, clip={clip}, correct_names={correct_names} ----")
                    wer, cer = evaluate(engine, orientation, clip, correct_names)
                    wer_list.append(wer)
                    cer_list.append(cer)

    return wer_list, cer_list

def plot_results(wer_list: List[float], cer_list: List[float]) -> None:
    fig, ax = plt.subplots()

    ocr_engines = ["doctr", "easyocr", "pytesseract"]
    orientations = [True, False]
    clips = [True, False]
    correct_names_options = [True, False]
    
    # Create a custom colormap
    cmap = mcolors.LinearSegmentedColormap.from_list(
    'pastel_rainbow', 
    #['#fca3b1', '#f7c1ad', '#ffdea7', '#e4f4a4', '#b2dfb5', '#82d7e6', '#8cb5e5', '#c0a5e8'],
    #['#FF9AA2', '#FFB7B2', '#FFDAC1', '#E2F0CB', '#B5EAD7', '#C7CEEA', '#B5B8FF'],
    ["#3F5D7D", "#F2B447", "#E76F51", "#2A9D8F"],
    N=len(wer_list))
    
    # Evaluate all combinations of the parameters
    for i, engine in enumerate(ocr_engines):
        for j, orientation in enumerate(orientations):
            for k, clip in enumerate(clips):
                for l, correct_names in enumerate(correct_names_options):
                    idx = i * len(orientations) * len(clips) * len(correct_names_options) \
                          + j * len(clips) * len(correct_names_options) \
                          + k * len(correct_names_options) \
                          + l
                    if idx >= len(wer_list):
                        continue
                    
                    label = f"{engine}, orientation={orientation}, clip={clip}, correct_names={correct_names}"
                    color = cmap(idx)
                    ax.scatter(wer_list[idx], cer_list[idx], label=label, color=color)
                    
    ax.set_xlabel("WER")
    ax.set_ylabel("CER")
    ax.set_title("OCR engine evaluation")
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_xticks(np.arange(0, 1.1, 0.1))
    ax.set_yticks(np.arange(0, 1.1, 0.1))
    ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=7)
    plt.show()

In [12]:
wer_list, cer_list = evaluate_all_combinations()
plot_results(wer_list, cer_list)

------ engine = doctr, orientation=True, clip=True, correct_names=True ----


------ engine = doctr, orientation=True, clip=True, correct_names=False ----


------ engine = doctr, orientation=True, clip=False, correct_names=True ----


: 

: 